# Swin Transformer Canonical Training + Phase 3 Export (Colab)

**Model:** Swin Transformer (Swin-Base via timm)

**Objective:** Replicate ResNet50 canonical pipeline with Swin:
- Phase 1: Canonical splits (load from Drive)
- Phase 2: Canonical classes (27 classes, fp=cdfa70b13f7390e6)
- Phase 3: Export contract (.npz + _meta.json) with strict validation

**Expected outputs:**
- `STORE/artifacts/exports/swin_canonical_smoke/val.npz`
- `STORE/artifacts/exports/swin_canonical_smoke/val_meta.json`

**Validation:**
- split_signature must match ResNet50: `cf53f8eb169b3531`
- classes_fp must equal canonical: `cdfa70b13f7390e6`
- idx order must align with ResNet50 for fusion compatibility

In [1]:
from pathlib import Path
import os

from google.colab import drive
drive.mount("/content/drive")

# --- EDIT THESE PATHS ONCE ---
DRIVE_CODE_SNAPSHOT = Path("/content/drive/MyDrive/DS_rakuten_colab")
DRIVE_STORE = Path("/content/drive/MyDrive/DS_rakuten_store")
DRIVE_SPLITS_SRC = DRIVE_STORE / "splits"   # expects train_idx.txt / val_idx.txt / test_idx.txt
# ----------------------------

assert DRIVE_CODE_SNAPSHOT.exists(), f"Missing code snapshot: {DRIVE_CODE_SNAPSHOT}"
DRIVE_STORE.mkdir(parents=True, exist_ok=True)

os.environ["DS_RAKUTEN_STORE"] = str(DRIVE_STORE)

print("✓ DRIVE_CODE_SNAPSHOT:", DRIVE_CODE_SNAPSHOT)
print("✓ DRIVE_STORE:", DRIVE_STORE)
print("✓ DRIVE_SPLITS_SRC:", DRIVE_SPLITS_SRC)

Mounted at /content/drive
✓ DRIVE_CODE_SNAPSHOT: /content/drive/MyDrive/DS_rakuten_colab
✓ DRIVE_STORE: /content/drive/MyDrive/DS_rakuten_store
✓ DRIVE_SPLITS_SRC: /content/drive/MyDrive/DS_rakuten_store/splits


In [2]:
import shutil
import sys
from pathlib import Path

RUNTIME_ROOT = Path("/content/DS_rakuten")

# Clean and copy for deterministic imports
if RUNTIME_ROOT.exists():
    shutil.rmtree(RUNTIME_ROOT)

shutil.copytree(DRIVE_CODE_SNAPSHOT, RUNTIME_ROOT)

sys.path.insert(0, str(RUNTIME_ROOT))

print("✓ Runtime code ready:", RUNTIME_ROOT)
print("✓ sys.path[0]:", sys.path[0])

✓ Runtime code ready: /content/DS_rakuten
✓ sys.path[0]: /content/DS_rakuten


In [3]:
from pathlib import Path
import shutil

runtime_splits_dir = Path("/content/DS_rakuten/data/splits")
runtime_splits_dir.mkdir(parents=True, exist_ok=True)

# Copy txt files from Drive persistent store into /content runtime repo
src_files = ["train_idx.txt", "val_idx.txt", "test_idx.txt"]
for fn in src_files:
    src = DRIVE_SPLITS_SRC / fn
    dst = runtime_splits_dir / fn
    assert src.exists(), f"Missing split file in Drive: {src}"
    shutil.copy2(src, dst)

print("✓ Splits synced to:", runtime_splits_dir)
print("✓ Contents:", list(runtime_splits_dir.glob("*.txt"))[:10])

✓ Splits synced to: /content/DS_rakuten/data/splits
✓ Contents: [PosixPath('/content/DS_rakuten/data/splits/test_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/val_idx.txt'), PosixPath('/content/DS_rakuten/data/splits/train_idx.txt')]


In [4]:
# Install timm for Swin Transformer models
!pip -q install timm wandb

# Uncomment if your session is missing other packages:
# !pip -q install gdown
# !pip -q install scikit-learn

import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xiaosong-dev (xiaosong-dev-formation-data-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
from pathlib import Path

IMAGE_FILE_ID = "15ZkS0iTQ7j3mHpxil4mABlXwP-jAN_zi"

BASE_DIR = Path("/content/images")
TMP_DIR = Path("/content/tmp")
ZIP_PATH = TMP_DIR / "images.zip"

BASE_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)

if not ZIP_PATH.exists():
    print("Downloading images zip...")
    !gdown --id $IMAGE_FILE_ID -O {str(ZIP_PATH)}
else:
    print("Zip already present:", ZIP_PATH)

print("Unzipping images...")
!unzip -q -o {str(ZIP_PATH)} -d {str(BASE_DIR)}

def count_jpgs(p: Path, limit: int = 2000) -> int:
    if not p.exists():
        return 0
    n = 0
    for _ in p.rglob("*.jpg"):
        n += 1
        if n >= limit:
            break
    return n

# Common candidates
candidates = [
    BASE_DIR / "images" / "image_train",
    BASE_DIR / "image_train",
    BASE_DIR / "images" / "images" / "image_train",
]

best = None
best_count = 0
for c in candidates:
    n = count_jpgs(c)
    if n > best_count:
        best, best_count = c, n

# Fallback: search any folder named image_train
if best_count == 0:
    for c in BASE_DIR.rglob("image_train"):
        if c.is_dir():
            n = count_jpgs(c)
            if n > best_count:
                best, best_count = c, n

assert best is not None and best_count > 0, (
    "Could not find an image_train directory with jpg files under /content/images. "
    "Check zip content and unzip path."
)

IMG_ROOT = best
sample_jpg = next(IMG_ROOT.rglob("*.jpg"))

print("✓ IMG_ROOT detected:", IMG_ROOT)
print("✓ sample jpg:", sample_jpg)

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=15ZkS0iTQ7j3mHpxil4mABlXwP-jAN_zi
From (redirected): https://drive.google.com/uc?id=15ZkS0iTQ7j3mHpxil4mABlXwP-jAN_zi&confirm=t&uuid=82ed5ad5-bc75-4db7-aaba-99f51990ff66
To: /content/tmp/images.zip
100% 2.56G/2.56G [00:21<00:00, 116MB/s]
Unzipping images...
✓ IMG_ROOT detected: /content/images/images/image_train
✓ sample jpg: /content/images/images/image_train/image_1010030825_product_443748930.jpg


In [6]:
from src.data.image_dataset import RakutenImageDataset
from src.train.image_swin import SwinConfig, run_swin_canonical

print("✓ RakutenImageDataset:", RakutenImageDataset)
print("✓ SwinConfig:", SwinConfig)
print("✓ run_swin_canonical:", run_swin_canonical)

✓ RakutenImageDataset: <class 'src.data.image_dataset.RakutenImageDataset'>
✓ SwinConfig: <class 'src.train.image_swin.SwinConfig'>
✓ run_swin_canonical: <function run_swin_canonical at 0x7feacbf8a3e0>


In [7]:
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=True)
sig = split_signature(splits)

print("✓ signature:", sig)
print({k: len(v) for k, v in splits.items()})

[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits
✓ signature: cf53f8eb169b3531
{'train_idx': 61351, 'val_idx': 10827, 'test_idx': 12738}


In [8]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])

wandb.init(
    project="rakuten_image",
    name="swin_base",
    config={
        "model": "swin_base_patch4_window7_224",
        "batch_size": 32,
        "lr": 5e-5,
        "epochs": 20,
    }
)

cfg = SwinConfig(
    raw_dir=str(STORE / "data_raw"),
    img_dir=str(IMG_ROOT),  # must be /content local disk for speed
    out_dir=str(STORE / "artifacts" / "exports"),
    ckpt_dir=str(STORE / "checkpoints" / "image_swin"),

    img_size=224,
    batch_size=256,
    num_workers=8,
    num_epochs=20,
    lr=5e-5,
    weight_decay=0.05,

    use_amp=True,
    label_smoothing=0.1,
    dropout_rate=0.5,
    head_dropout2=0.3,
    drop_path_rate=0.3,

    mixup_alpha=0.8,
    cutmix_alpha=1.0,

    swin_model_name="swin_base_patch4_window7_224",
    swin_pretrained=True,

    force_colab_loader=True,  # Force Colab data loader

    model_name="swin_canonical_smoke",
    export_split="val",
)

wandb.config.update(cfg.__dict__)

try:

    result = run_swin_canonical(cfg)

    print("EXPORT:", result["export_result"])
    print("VERIFY:", result["verify_metadata"])
    print("probs_shape:", result["probs_shape"])
    print("best_val_f1:", result["best_val_f1"])

    wandb.log({"best_val_f1": result["best_val_f1"]})

finally:
    wandb.finish()

[INFO] Using Colab data loader (forced via force_colab_loader=True)
[load_data_colab] raw_dir: /content/drive/MyDrive/DS_rakuten_store/data_raw
[load_data_colab] img_root: /content/images/images/image_train
[load_data_colab] X: /content/drive/MyDrive/DS_rakuten_store/data_raw/X_train_update.csv
[load_data_colab] Y: /content/drive/MyDrive/DS_rakuten_store/data_raw/Y_train_CVw08PX.csv
[split_manager] Loading canonical splits from /content/DS_rakuten/data/splits


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Epoch 1/20 | train_loss=2.5583 train_f1=0.0000 | val_loss=1.3987 val_f1=0.4898 | lr=4.97e-05


Epoch 2/20 | train_loss=2.2124 train_f1=0.0000 | val_loss=1.2050 val_f1=0.5752 | lr=4.88e-05


...

Epoch 18/20 | train_loss=1.7431 train_f1=0.0000 | val_loss=0.9169 val_f1=0.7165 | lr=2.20e-06


Epoch 19/20 | train_loss=1.7313 train_f1=0.0000 | val_loss=0.9150 val_f1=0.7172 | lr=1.30e-06


Epoch 20/20 | train_loss=1.7151 train_f1=0.0000 | val_loss=0.9164 val_f1=0.7163 | lr=1.00e-06


[OK] Exported model=swin_canonical_smoke split=val npz=/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/swin_canonical_smoke/val.npz sig=cf53f8eb169b3531 fp=cdfa70b13f7390e6 n=10827
EXPORT: {'npz_path': '/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/swin_canonical_smoke/val.npz', 'meta_json_path': '/content/drive/MyDrive/DS_rakuten_store/artifacts/exports/swin_canonical_smoke/val_meta.json', 'classes_fp': 'cdfa70b13f7390e6', 'split_signature': 'cf53f8eb169b3531', 'num_samples': 10827}
VERIFY: {'model_name': 'swin_canonical_smoke', 'split_name': 'val', 'split_signature': 'cf53f8eb169b3531', 'classes_fp': 'cdfa70b13f7390e6', 'num_classes': 27, 'num_samples': 10827, 'has_y_true': True, 'probs_shape': [10827, 27], 'probs_dtype': 'float32', 'created_at': '2026-01-09T18:30:09.158872', 'extra': {'source': 'src/train/image_swin.py', 'model_architecture': 'timm.swin_base_patch4_window7_224', 'swin_pretrained': True, 'img_dir': '/content/images/images/image_train', 'img_size

best_val_f1,▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,████▇▇▇▆▆▅▄▄▃▃▂▂▂▁▁▁
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_f1,▁▄▅▆▆▇▇▇▇▇█▇████████
best_val_f1,0.71716
epoch,19
lr,0.0
train_acc,0


In [9]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
export_dir = STORE / "artifacts" / "exports" / "swin_canonical_smoke"

print("Export dir:", export_dir)
print("Contents:", [p.name for p in export_dir.glob("*")])

assert (export_dir / "val.npz").exists(), "Missing val.npz"
assert (export_dir / "val_meta.json").exists(), "Missing val_meta.json"
print("✓ Export files exist.")

Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/swin_canonical_smoke
Contents: ['val_meta.json', 'val.npz']
✓ Export files exist.


In [10]:
!python -m apps.image_app.scripts.validate_exports --split val --exports-root "$DS_RAKUTEN_STORE/artifacts/exports" --strict

/usr/bin/python3: Error while finding module specification for 'apps.image_app.scripts.validate_exports' (ModuleNotFoundError: No module named 'apps')


In [11]:
import json
from pathlib import Path
import os

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
meta_path = STORE / "artifacts" / "exports" / "swin_canonical_smoke" / "val_meta.json"

meta = json.loads(meta_path.read_text())
keys = [
    "model_name", "split_name", "split_signature",
    "classes_fp", "num_samples", "probs_shape"
]
for k in keys:
    print(f"{k}: {meta.get(k)}")

model_name: swin_canonical_smoke
split_name: val
split_signature: cf53f8eb169b3531
classes_fp: cdfa70b13f7390e6
num_samples: 10827
probs_shape: [10827, 27]


In [12]:
import shutil
from pathlib import Path
from src.export.model_exporter import load_predictions
from src.data.label_mapping import CANONICAL_CLASSES_FP
from src.data.split_manager import load_splits, split_signature

splits = load_splits(verbose=False)
sig = split_signature(splits)

CACHE = Path("/content/cache_exports")
CACHE.mkdir(parents=True, exist_ok=True)

export_result = result["export_result"]
npz_src = Path(export_result["npz_path"])
meta_src = npz_src.with_name(npz_src.stem + "_meta.json")

npz_local = CACHE / npz_src.name
meta_local = CACHE / meta_src.name

# Copy both files (npz + meta)
if (not npz_local.exists()) or (npz_local.stat().st_size != npz_src.stat().st_size):
    shutil.copy2(npz_src, npz_local)

if (not meta_local.exists()) or (meta_local.stat().st_size != meta_src.stat().st_size):
    shutil.copy2(meta_src, meta_local)

loaded = load_predictions(
    npz_path=str(npz_local),
    verify_split_signature=sig,
    verify_classes_fp=CANONICAL_CLASSES_FP,
    require_y_true=True,
)

print("✓ loaded ok")
print("model:", loaded["metadata"]["model_name"])
print("split:", loaded["metadata"]["split_name"])
print("sig:", loaded["metadata"]["split_signature"])
print("fp:", loaded["metadata"]["classes_fp"])
print("probs:", loaded["probs"].shape)

✓ loaded ok
model: swin_canonical_smoke
split: val
sig: cf53f8eb169b3531
fp: cdfa70b13f7390e6
probs: (10827, 27)


In [13]:
import os
from pathlib import Path

STORE = Path(os.environ["DS_RAKUTEN_STORE"])
export_dir = STORE / "artifacts" / "exports" / "swin_canonical_smoke"

print("Export dir:", export_dir)
print("Files:", [p.name for p in export_dir.glob("*")])

assert (export_dir / "val.npz").exists(), "Missing val.npz"
assert (export_dir / "val_meta.json").exists(), "Missing val_meta.json"
print("✓ Export files exist")

Export dir: /content/drive/MyDrive/DS_rakuten_store/artifacts/exports/swin_canonical_smoke
Files: ['val_meta.json', 'val.npz']
✓ Export files exist
